In [11]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [12]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt

import lightgbm as lgbm
import gc
import xgboost as xgb
import pickle as pickle


from catboost import CatBoostRegressor
import dask.dataframe as dd
from sklearn.model_selection import KFold
from itertools import product

In [13]:
items           = pd.read_csv('items.csv',usecols=["item_id", "item_category_id"])
item_categories = pd.read_csv('item_categories.csv')
shops           = pd.read_csv('shops.csv')
sales_train     = pd.read_csv('sales_train.csv.gz')
test            = pd.read_csv('test.csv.gz')

In [14]:
sales_train[['day','month', 'year']] = sales_train['date'].str.split('.', expand=True).astype(int)
sales_train = sales_train[sales_train['year'] != 2013]
sales_train = sales_train.set_index('item_id').join(items.set_index('item_id'))
sales_train.reset_index(inplace=True)

In [15]:
train_item_ids = sales_train['item_id'].unique()
train_shop_ids = sales_train['shop_id'].unique()
test_item_ids = test['item_id'].unique()
test_shop_ids = test['shop_id'].unique()
train_blocks = sales_train['date_block_num'].unique()

all_item_ids = np.unique(np.append(test_item_ids,train_item_ids))
all_shop_ids = np.unique(np.append(train_shop_ids,test_shop_ids))

In [16]:
combinations = []

for dbn in range(np.min(train_blocks), np.max(train_blocks)+1):
    sales = sales_train[sales_train.date_block_num==dbn]
    dbn_combos = list(product(sales.shop_id.unique(), sales.item_id.unique(), [dbn]))
    for combo in dbn_combos:
        combinations.append(combo)
        
all_combos = pd.DataFrame(np.unique(np.vstack([combinations]), axis=0), columns=['shop_id','item_id','date_block_num'])

In [17]:
ys = sales_train.groupby(['shop_id', 'item_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"item_cnt_block"})

training = all_combos.merge(ys, on=['shop_id', 'item_id', 'date_block_num'], how='left').fillna(0)


training['item_cnt_block'] = training['item_cnt_block'].clip(0,20).astype('int8')

training = training.set_index('item_id').join(items.set_index('item_id'))
training.reset_index(inplace=True)

for col in ['item_id', 'shop_id', 'item_category_id']:
    training[col] = pd.to_numeric(training[col], downcast='unsigned')

In [18]:
dates = sales_train[['date_block_num', 'month', 'year']].drop_duplicates(['date_block_num', 'month', 'year'])

dates_dict = {}

for index,row in dates.iterrows():
    dates_dict[row['date_block_num']] = {"month": row['month'], "year": row['year']}
    
training['month'] = pd.to_numeric(training['date_block_num'].apply(lambda block: dates_dict[block]['month']), downcast='unsigned')


In [19]:
cols = ['item_id','date_block_num']

training.groupby(cols,as_index=False)['item_cnt_block'].mean().sample(10)

,item_id,date_block_num,item_cnt_block
127311,20644,22,0.080000
121392,19668,31,0.047619
91719,15226,18,1.100000
61619,10539,18,0.020000
47145,8095,15,0.489796
16174,2920,28,0.022727
127156,20613,32,1.837209
39549,6709,19,0.078431
80707,13553,30,0.046512
104007,16960,24,0.060000


In [20]:
def add_block_units_mean(df, cols, name):
    print(name)
    name_units = name + '_units'
    name_mean = name + '_mean'
    
    try:
        df.drop(columns=[name_units, name_mean],inplace=True)
    except:
        pass

    
    block_units = df.groupby(cols,as_index=False)['item_cnt_block'].sum()\
                        .drop_duplicates(cols)\
                        .rename(columns={'item_cnt_block':name_units})
    df = df.merge(block_units, on=cols, how='left')
    df[name_units].fillna(0,inplace=True)
    df[name_units] = pd.to_numeric(df[name_units].astype(int),downcast='unsigned')
    del block_units
    
    block_means = df.groupby(cols,as_index=False)['item_cnt_block'].mean()\
                        .drop_duplicates(cols)\
                        .rename(columns={'item_cnt_block':name_mean})
    df = df.merge(block_means, on=cols, how='left')
    df[name_mean].fillna(0,inplace=True)
    df[name_mean] = pd.to_numeric(df[name_mean],downcast='float')
    del block_means
    
    gc.collect()
    return df


training = add_block_units_mean(training, ['item_id','date_block_num'], 'item_block')
training = add_block_units_mean(training, ['shop_id','date_block_num'], 'shop_block')
training = add_block_units_mean(training, ['item_category_id','date_block_num'], 'cat_block')
training = add_block_units_mean(training, ['shop_id', 'item_category_id','date_block_num'], 'shop_cat_block')
training = add_block_units_mean(training, ['shop_id', 'item_id','date_block_num'], 'shop_item_block')

item_block
shop_block
cat_block
shop_cat_block
shop_item_block


In [218]:
number_of_items = sales_train['item_id'].nunique()
print("number_of_items:", number_of_items)
number_of_categories = sales_train['item_category_id'].nunique()
print("number_of_categories:", number_of_categories)
number_of_shops = sales_train['shop_id'].nunique()
print("number_of_shops:", number_of_shops)
number_of_days = 365 + 365 - 30 - 31
print("number_of_days:", number_of_days)
number_of_blocks = sales_train['date_block_num'].nunique()
print("number_of_blocks:", number_of_blocks)
total_sales = sales_train['item_cnt_day'].sum()
print("total_sales:", total_sales)
average_price = sales_train['item_price'].mean()
print("average_price:", average_price)

training['item_units'] = pd.to_numeric(training.groupby(['date_block_num'])['item_block_units'].transform(np.sum),downcast='unsigned')
training['cat_units'] = pd.to_numeric(training.groupby(['date_block_num'])['cat_block_units'].transform(np.sum),downcast='unsigned')
training['shop_units'] = pd.to_numeric(training.groupby(['date_block_num'])['shop_block_units'].transform(np.sum),downcast='unsigned')

training['item_share_of_total_units'] = pd.to_numeric(training['item_units'] * 100 / total_sales,downcast='float')
training['category_share_of_total_units'] = pd.to_numeric(training['cat_units'] * 100 / total_sales,downcast='float')
training['shop_share_of_units'] = pd.to_numeric(training['shop_units'] * 100 / total_sales,downcast='float')
training['shop_item_units'] = pd.to_numeric(training.groupby(['date_block_num'])\
                                            ['shop_item_block_units'].transform(np.sum),downcast='unsigned')

training['shop_item_share_of_total_units'] = pd.to_numeric(training['shop_item_units'] * 100\
                        / total_sales,downcast='float')
training['shop_item_share_of_shop_units'] = pd.to_numeric(training['shop_item_units'] * 100\
                        / training['shop_units'],downcast='float')


training['item_share_of_shop_units'] = pd.to_numeric(training['shop_item_units'] * 100 / training['shop_units'],downcast='float')

training['shop_item_share_of_shop_units_mean'] = training.groupby('item_id')['shop_item_share_of_shop_units'].transform(np.mean)


number_of_items: 17054
number_of_categories: 79
number_of_shops: 55
number_of_days: 669
number_of_blocks: 22
total_sales: 2085473.0
average_price: 1015.5023073772021


In [23]:
def get_mean_encoding(df, group_cols, target):
    cumsum = df.groupby(group_cols)[target].cumsum() - df[target]
    cumcnt = df.groupby(group_cols).cumcount()
    return pd.to_numeric(cumsum/cumcnt, downcast='float')

training['item_me'] = get_mean_encoding(training, ['item_id'], 'item_cnt_block')
training['shop_me'] = get_mean_encoding(training, ['shop_id'], 'item_cnt_block')
training['category_me'] = get_mean_encoding(training, ['item_category_id'], 'item_cnt_block')
training['shop_category_me'] = get_mean_encoding(training, ['shop_id', 'item_category_id'], 'item_cnt_block')
training['shop_item_me'] = get_mean_encoding(training, ['shop_id', 'item_id'], 'item_cnt_block')
training['month_me'] = get_mean_encoding(training, ['month'], 'item_cnt_block')
training['block_me'] = get_mean_encoding(training, ['date_block_num'], 'item_cnt_block')



training.fillna(0,inplace=True)

In [24]:
def add_min_max_quantiles(df, cols, name):
    print(name)

    block_name = name+'_block_units'
    units_name = name+'_units'
    max_name = name+'_max_units_block'
    min_name = name+'_min_units_block'
    
    try:
        df.drop(columns=[units_name, max_name, min_name, min_max_name],inplace=True)
    except:
        pass


    df[units_name] = pd.to_numeric(df.groupby(['date_block_num'])[block_name].transform(np.sum), downcast='unsigned')
    df[max_name] = pd.to_numeric(df.groupby(cols)[block_name].transform(np.max), downcast='unsigned')
    df[min_name] = pd.to_numeric(df.groupby(cols)[block_name].transform(np.min), downcast='unsigned')
    


    for q in [0.25,0.50,0.75]:
        qname = name+'_minmax_q' + str(q)
        try:
            df.drop(columns=[qname],inplace=True)
        except:
            pass
        df[qname] =  pd.to_numeric(df[[min_name,max_name]].quantile(q,axis=1), downcast='unsigned')
        
    return df

training = add_min_max_quantiles(training, ['item_id'], 'item')
training = add_min_max_quantiles(training, ['shop_id'], 'shop')
training = add_min_max_quantiles(training, ['item_category_id'], 'cat')
training = add_min_max_quantiles(training, ['shop_id','item_category_id'], 'shop_cat')
training = add_min_max_quantiles(training, ['shop_id','item_id'], 'shop_item')

item
shop
cat
shop_cat
shop_item


In [25]:
def add_rolls(df, cols, name, rolls = [3]):
    for roll in rolls:
        print(name, roll)
        roll_name = name+"_rolling_" + str(roll)
        roll_name_tmp = roll_name + "_tmp"
        
        try:
            df.drop(columns=[roll_name],inplace=True)
        except:
            pass       

    
        block_units_rolling_temp = df\
            .drop_duplicates(cols)\
            .sort_values(cols)\
            .set_index(cols)\
            .groupby(cols[0:len(cols)-1],as_index=False)\
            [name].rolling(roll,min_periods=2).mean().reset_index()\
            .rename(columns={name:roll_name_tmp})\
            [cols+[roll_name_tmp]]
        
    
        df = df.merge(block_units_rolling_temp, on=cols, how='left')
        #print(df.columns.values)
        del block_units_rolling_temp
        gc.collect()
        

        block_units_rolling = df\
            .drop_duplicates(cols)\
            .sort_values(cols)\
            .set_index(cols)\
            .groupby(cols[0:len(cols)-1],as_index=False)\
            [roll_name_tmp].shift(1)\
            .rename(columns={roll_name_tmp:roll_name}).reset_index()

        df = df.merge(block_units_rolling, on=cols, how='left')
        df[roll_name].fillna(0,inplace=True)
        df[roll_name] = pd.to_numeric(df[roll_name], downcast='float')
        df.drop(columns=[roll_name_tmp], inplace=True)
        del block_units_rolling
        gc.collect()
    
    return df
    

training = add_rolls(training, ['item_id','date_block_num'], 'item_block_units')
training = add_rolls(training, ['item_id','date_block_num'], 'item_block_mean')
training = add_rolls(training, ['shop_id','date_block_num'], 'shop_block_units')
training = add_rolls(training, ['shop_id','date_block_num'], 'shop_block_mean')
training = add_rolls(training, ['item_category_id','date_block_num'], 'cat_block_units')
training = add_rolls(training, ['item_category_id','date_block_num'], 'cat_block_mean')
training = add_rolls(training, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_units')
training = add_rolls(training, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_mean')
#training = add_rolls(training, ['shop_id','item_id','date_block_num'], 'shop_item')

item_block_units 3
item_block_mean 3
shop_block_units 3
shop_block_mean 3
cat_block_units 3
cat_block_mean 3
shop_cat_block_units 3
shop_cat_block_mean 3


In [26]:
training = add_rolls(training, ['shop_id','item_id','date_block_num'], 'shop_item_block_mean')

shop_item_block_mean 3


In [27]:
def add_lags(df, cols, name, lags = [1]):
    
    for lag in lags:
        print(name, lag)
        lag_name = name + "_lag_" + str(lag)
        
        try:
            df.drop(columns=[lag_name],inplace=True)
        except:
            pass       

        result = df\
            .drop_duplicates(cols)\
            .sort_values(cols)\
            .set_index(cols)\
            .groupby(cols[0:len(cols)-1],as_index=False)\
            [name].shift(lag)\
            .rename(columns={name:lag_name}).reset_index()

        df = df.merge(result, on=cols, how='left')
        df[lag_name].fillna(0,inplace=True)
        if "mean" in name:
            df[lag_name] = pd.to_numeric(df[lag_name], downcast='float')
        else:
            df[lag_name] = pd.to_numeric(df[lag_name].astype(int), downcast='unsigned')
        del result
        gc.collect()
    
    return df
                                         

                                        
training = add_lags(training, ['item_id','date_block_num'], 'item_block_units')
training = add_lags(training, ['item_id','date_block_num'], 'item_block_mean')
training = add_lags(training, ['shop_id','date_block_num'], 'shop_block_units')
training = add_lags(training, ['shop_id','date_block_num'], 'shop_block_mean')
training = add_lags(training, ['item_category_id','date_block_num'], 'cat_block_units')
training = add_lags(training, ['item_category_id','date_block_num'], 'cat_block_mean')
training = add_lags(training, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_units')
training = add_lags(training, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_mean')
training = add_lags(training, ['shop_id','item_id','date_block_num'], 'shop_item_block_units')
training = add_lags(training, ['shop_id','item_id','date_block_num'], 'shop_item_block_mean')

item_block_units 1
item_block_mean 1
shop_block_units 1
shop_block_mean 1
cat_block_units 1
cat_block_mean 1
shop_cat_block_units 1
shop_cat_block_mean 1
shop_item_block_units 1
shop_item_block_mean 1


In [28]:
training['shop_block_units_lag_comp1'] = pd.to_numeric(training['shop_block_units_lag_1'] * training['item_share_of_shop_units'],downcast='unsigned')

#training['shop_share_item_units_comp'] = training['item_units'] * training['shop_share_of_units']
training['item_block_units_lag_comp1'] = pd.to_numeric(training['item_block_units_lag_1'] * training['item_share_of_shop_units'],downcast='unsigned')

In [29]:
training.columns.values

array(['item_id', 'shop_id', 'date_block_num', 'item_cnt_block',
       'item_category_id', 'month', 'item_block_units', 'item_block_mean',
       'shop_block_units', 'shop_block_mean', 'cat_block_units',
       'cat_block_mean', 'shop_cat_block_units', 'shop_cat_block_mean',
       'shop_item_block_units', 'shop_item_block_mean', 'item_units',
       'cat_units', 'shop_units', 'item_share_of_total_units',
       'category_share_of_total_units', 'shop_share_of_units',
       'shop_item_units', 'shop_item_share_of_total_units',
       'shop_item_share_of_shop_units', 'item_share_of_shop_units',
       'item_me', 'shop_me', 'category_me', 'shop_category_me',
       'shop_item_me', 'month_me', 'block_me', 'item_max_units_block',
       'item_min_units_block', 'item_minmax_q0.25', 'item_minmax_q0.5',
       'item_minmax_q0.75', 'shop_max_units_block',
       'shop_min_units_block', 'shop_minmax_q0.25', 'shop_minmax_q0.5',
       'shop_minmax_q0.75', 'cat_max_units_block', 'cat_min_units_bl

In [30]:

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
training.sample(10)

,item_id,shop_id,date_block_num,item_cnt_block,item_category_id,month,item_block_units,item_block_mean,shop_block_units,shop_block_mean,cat_block_units,cat_block_mean,shop_cat_block_units,shop_cat_block_mean,shop_item_block_units,shop_item_block_mean,item_units,cat_units,shop_units,item_share_of_total_units,category_share_of_total_units,shop_share_of_units,shop_item_units,shop_item_share_of_total_units,shop_item_share_of_shop_units,item_share_of_shop_units,item_me,shop_me,category_me,shop_category_me,shop_item_me,month_me,block_me,item_max_units_block,item_min_units_block,item_minmax_q0.25,item_minmax_q0.5,item_minmax_q0.75,shop_max_units_block,shop_min_units_block,shop_minmax_q0.25,shop_minmax_q0.5,shop_minmax_q0.75,cat_max_units_block,cat_min_units_block,cat_minmax_q0.25,cat_minmax_q0.5,cat_minmax_q0.75,shop_cat_units,shop_cat_max_units_block,shop_cat_min_units_block,shop_cat_minmax_q0.25,shop_cat_minmax_q0.5,shop_cat_minmax_q0.75,shop_item_max_units_block,shop_item_min_units_block,shop_item_minmax_q0.25,shop_item_minmax_q0.5,shop_item_minmax_q0.75,item_block_units_rolling_3,item_block_mean_rolling_3,shop_block_units_rolling_3,shop_block_mean_rolling_3,cat_block_units_rolling_3,cat_block_mean_rolling_3,shop_cat_block_units_rolling_3,shop_cat_block_mean_rolling_3,shop_item_block_mean_rolling_3,item_block_units_lag_1,item_block_mean_lag_1,shop_block_units_lag_1,shop_block_mean_lag_1,cat_block_units_lag_1,cat_block_mean_lag_1,shop_cat_block_units_lag_1,shop_cat_block_mean_lag_1,shop_item_block_units_lag_1,shop_item_block_mean_lag_1,shop_block_units_lag_comp1,item_block_units_lag_comp1
3453549,12206,22,18,0,62,7,6,0.120000,1054,0.159431,616,0.133913,6,0.065217,0,0,4209850,2140570050,556626367,201.865479,1727.956909,1977.023560,84197,4.037310,0.015126,0.015126,0.135294,0.253504,0.094121,0.060194,0.000000,0.285945,0.290972,18,2,6.00,10.0,14.00,2197,944,1257.25,1570.5,1883.75,1324,73,385.75,698.5,1011.25,42811401,18,0,4.50,9.0,13.50,1,0,0.25,0.5,0.75,11.000000,0.224490,1327.666626,0.197630,644.666687,0.151037,3.333333,0.038619,0.000000,12,0.244898,1163,0.174076,642,0.154142,0,0.000000,0,0.0,17.591892,0.181516
5981223,20391,5,18,0,72,7,4,0.080000,1290,0.195129,1649,0.227448,20,0.137931,0,0,4209850,2140570050,556626367,201.865479,1727.956909,1977.023560,84197,4.037310,0.015126,0.015126,0.100000,0.194829,0.183988,0.127590,0.333333,0.256405,0.254291,21,1,6.00,11.0,16.00,1953,875,1144.50,1414.0,1683.50,3970,1122,1834.00,2546.0,3258.00,42811401,55,11,22.00,33.0,44.00,1,0,0.25,0.5,0.75,3.000000,0.061224,1153.333374,0.171895,1878.000000,0.257740,28.333334,0.190273,0.000000,4,0.081633,1286,0.192486,1790,0.228316,24,0.150000,0,0.0,19.452427,0.060505
988960,3676,41,24,1,23,1,86,1.720000,1309,0.213227,5537,0.651412,88,0.517647,1,1,4871650,1709982800,598141187,233.599289,1675.665649,1908.221191,97433,4.671986,0.016289,0.016289,1.380368,0.251869,0.622957,0.490285,1.250000,0.399613,0.372121,166,12,50.50,89.0,127.50,1936,634,959.50,1285.0,1610.50,9520,2182,4016.50,5851.0,7685.50,34199656,166,31,64.75,98.5,132.25,4,0,1.00,2.0,3.00,100.666664,2.003333,1209.000000,0.187588,6221.666504,0.762720,78.333336,0.481344,1.000000,166,3.320000,1577,0.238759,9520,1.120000,134,0.788235,2,2.0,25.688223,2.704023
3922400,13888,31,30,2,55,7,20,0.465116,5392,1.012963,6475,0.216664,599,0.861871,2,2,2551405,795165761,315840205,122.341789,1058.352173,728.491516,59335,2.845158,0.018786,0.018786,1.749460,1.121119,0.232753,1.006835,8.437500,0.277544,0.272633,229,13,67.00,121.0,175.00,11884,5124,6814.00,8504.0,10194.00,13694,4917,7111.25,9305.5,11499.75,18492227,1413,522,744.75,967.5,1190.25,20,2,6.50,11.0,15.50,16.000000,0.357091,5563.333496,1.044428,6509.333496,0.209156,710.333313,1.018909,2.333333,15,0.348837,5507,1.055790,6017,0.207304,647,0.958519,3,3.0,103.456696,0.281796
1342100,4859,24,21,0,76,10,3,0.057692,1407,0.222135,226,0.062988,0,0.000000,0,0,4801004,1766935664,584799218,230.211761,287.660034,1268.463745,92327,4.427149,0.015788,0.015788,0.000000,0.409206,0.063

In [109]:
training.columns.values

array(['item_id', 'shop_id', 'date_block_num', 'item_cnt_block',
       'item_category_id', 'month', 'item_block_units', 'item_block_mean',
       'shop_block_units', 'shop_block_mean', 'cat_block_units',
       'cat_block_mean', 'shop_cat_block_units', 'shop_cat_block_mean',
       'shop_item_block_units', 'shop_item_block_mean', 'item_units',
       'cat_units', 'shop_units', 'item_share_of_total_units',
       'category_share_of_total_units', 'shop_share_of_units',
       'shop_item_units', 'shop_item_share_of_total_units',
       'shop_item_share_of_shop_units', 'item_share_of_shop_units',
       'item_me', 'shop_me', 'category_me', 'shop_category_me',
       'shop_item_me', 'month_me', 'block_me', 'item_max_units_block',
       'item_min_units_block', 'item_minmax_q0.25', 'item_minmax_q0.5',
       'item_minmax_q0.75', 'shop_max_units_block',
       'shop_min_units_block', 'shop_minmax_q0.25', 'shop_minmax_q0.5',
       'shop_minmax_q0.75', 'cat_max_units_block', 'cat_min_units_bl

In [219]:
gc.collect()

ZEROS_KEEP=0.2


#x_train = training[(training['date_block_num'] < 33) & (training['val_ignore'] == False)]
x_train = training[(training['date_block_num'] < 33)]
y_train = x_train['item_cnt_block']





x_val = training[training['date_block_num'] == 33]
y_val = x_val['item_cnt_block']

pos_val_len = len(y_val[y_val != 0])
print("pos_val_len", pos_val_len)

zeros_keep_indices_val = y_val[y_val == 0].sample(int(pos_val_len/ZEROS_KEEP)).index
print("zeros_keep_indices_val", len(zeros_keep_indices_val))
non_zeros_val_indices = y_val[y_val != 0].index
print("non_zeros_val_indices", len(non_zeros_val_indices))

val_indices = np.append(np.array(zeros_keep_indices_val), np.array(non_zeros_val_indices))

y_val = y_val.loc[val_indices]
x_val = x_val.loc[val_indices]



pos_val_len 31471
zeros_keep_indices_val 157355
non_zeros_val_indices 31471


In [110]:
features = [
    
    'item_units',
       'cat_units', 'shop_units', 'item_share_of_total_units',
       'category_share_of_total_units', 'shop_share_of_units',
       'shop_item_units', 'shop_item_share_of_total_units',
       'shop_item_share_of_shop_units', 'item_share_of_shop_units',
       'item_me', 'shop_me', 'category_me', 'shop_category_me',
       'shop_item_me', 'month_me', 'block_me', 'item_max_units_block',
       'item_min_units_block', 'item_minmax_q0.25', 'item_minmax_q0.5',
       'item_minmax_q0.75', 'shop_max_units_block',
       'shop_min_units_block', 'shop_minmax_q0.25', 'shop_minmax_q0.5',
       'shop_minmax_q0.75', 'cat_max_units_block', 'cat_min_units_block',
       'cat_minmax_q0.25', 'cat_minmax_q0.5', 'cat_minmax_q0.75',
       'shop_cat_units', 'shop_cat_max_units_block',
       'shop_cat_min_units_block', 'shop_cat_minmax_q0.25',
       'shop_cat_minmax_q0.5', 'shop_cat_minmax_q0.75',
       'shop_item_max_units_block', 'shop_item_min_units_block',
       'shop_item_minmax_q0.25', 'shop_item_minmax_q0.5',
       'shop_item_minmax_q0.75', 'item_block_units_rolling_3',
       'item_block_mean_rolling_3', 'shop_block_units_rolling_3',
       'shop_block_mean_rolling_3', 'cat_block_units_rolling_3',
       'cat_block_mean_rolling_3', 'shop_cat_block_units_rolling_3',
       'shop_cat_block_mean_rolling_3', 'shop_item_block_mean_rolling_3',
       'item_block_units_lag_1', 'item_block_mean_lag_1',
       'shop_block_units_lag_1', 'shop_block_mean_lag_1',
       'cat_block_units_lag_1', 'cat_block_mean_lag_1',
       'shop_cat_block_units_lag_1', 'shop_cat_block_mean_lag_1',
       'shop_item_block_units_lag_1', 'shop_item_block_mean_lag_1',
       'shop_block_units_lag_comp1', 'item_block_units_lag_comp1',
    'shop_item_share_of_shop_units_mean'


]

In [234]:

features = [
    
    
        
       'item_block_mean_rolling_3',
       'shop_block_mean_rolling_3',
           'shop_cat_block_mean_rolling_3',



      'item_block_mean_lag_1',
        'shop_block_mean_lag_1',
            'shop_cat_block_mean_lag_1',
    
    'shop_item_share_of_shop_units_mean',
    'shop_item_block_mean_rolling_3',
    'shop_item_block_mean_lag_1'
]




In [235]:
cb_model = CatBoostRegressor(iterations=1000,
                             #learning_rate=0.05,
                             eval_metric='RMSE',
                             task_type = "GPU",
                             use_best_model=True,
                             od_type = "Iter",
                             od_wait = 1,
                             bagging_temperature = 30,
                             #cat_features=[0],
                             random_seed = 42)

#drops = ['subcategory','area']
#x_train = x_train.drop(columns=drops)
#x_val = x_val.drop(columns=drops)


cb_model.fit(x_train[features], y_train, #cat_features=categorical_features_indices,
             eval_set=(x_val[features],y_val),
             #cat_features=categorical_features_pos,         
             verbose=True)

scores = {}
for i,score in enumerate(cb_model.get_feature_importance()):
    scores[features[i]] = score

sorted(scores.items(), key=lambda x: x[1])[::-1]

0:	learn: 1.2133980	test: 1.2999837	best: 1.2999837 (0)	total: 44.1ms	remaining: 44s
1:	learn: 1.2029546	test: 1.2903748	best: 1.2903748 (1)	total: 86.3ms	remaining: 43.1s
2:	learn: 1.1985296	test: 1.2861497	best: 1.2861497 (2)	total: 132ms	remaining: 43.8s
3:	learn: 1.1919853	test: 1.2797796	best: 1.2797796 (3)	total: 177ms	remaining: 44.1s
4:	learn: 1.1838821	test: 1.2724153	best: 1.2724153 (4)	total: 222ms	remaining: 44.1s
5:	learn: 1.1782269	test: 1.2666749	best: 1.2666749 (5)	total: 267ms	remaining: 44.2s
6:	learn: 1.1705877	test: 1.2579635	best: 1.2579635 (6)	total: 309ms	remaining: 43.8s
7:	learn: 1.1630622	test: 1.2506970	best: 1.2506970 (7)	total: 350ms	remaining: 43.5s
8:	learn: 1.1545329	test: 1.2436183	best: 1.2436183 (8)	total: 391ms	remaining: 43.1s
9:	learn: 1.1467970	test: 1.2365891	best: 1.2365891 (9)	total: 425ms	remaining: 42s
10:	learn: 1.1403980	test: 1.2309178	best: 1.2309178 (10)	total: 467ms	remaining: 42s
11:	learn: 1.1368427	test: 1.2257458	best: 1.2257458 (11

97:	learn: 0.9621252	test: 1.0654513	best: 1.0654513 (97)	total: 4.08s	remaining: 37.6s
98:	learn: 0.9602945	test: 1.0639080	best: 1.0639080 (98)	total: 4.13s	remaining: 37.6s
99:	learn: 0.9588460	test: 1.0625684	best: 1.0625684 (99)	total: 4.17s	remaining: 37.5s
100:	learn: 0.9578313	test: 1.0618800	best: 1.0618800 (100)	total: 4.21s	remaining: 37.5s
101:	learn: 0.9569469	test: 1.0609047	best: 1.0609047 (101)	total: 4.25s	remaining: 37.4s
102:	learn: 0.9565364	test: 1.0605685	best: 1.0605685 (102)	total: 4.29s	remaining: 37.3s
103:	learn: 0.9563973	test: 1.0604500	best: 1.0604500 (103)	total: 4.33s	remaining: 37.3s
104:	learn: 0.9556856	test: 1.0601106	best: 1.0601106 (104)	total: 4.37s	remaining: 37.2s
105:	learn: 0.9551347	test: 1.0596896	best: 1.0596896 (105)	total: 4.41s	remaining: 37.2s
106:	learn: 0.9550296	test: 1.0595972	best: 1.0595972 (106)	total: 4.46s	remaining: 37.2s
107:	learn: 0.9547438	test: 1.0592788	best: 1.0592788 (107)	total: 4.5s	remaining: 37.2s
108:	learn: 0.954

[('item_block_mean_lag_1', 28.117482527581426),
 ('shop_item_block_mean_lag_1', 19.21500226904894),
 ('shop_item_block_mean_rolling_3', 14.695665861041663),
 ('shop_cat_block_mean_lag_1', 9.62481388554665),
 ('shop_cat_block_mean_rolling_3', 9.081632843280458),
 ('item_block_mean_rolling_3', 7.144243372343445),
 ('shop_item_share_of_shop_units_mean', 6.150263187507847),
 ('shop_block_mean_lag_1', 3.052541194425141),
 ('shop_block_mean_rolling_3', 2.918354859224435)]

In [48]:
features = [item[0] for item in scores.items() if item[1] > 2000]

In [189]:
test            = pd.read_csv('test.csv.gz')
test = test.set_index('item_id').join(items.set_index('item_id'))
test.reset_index(inplace=True)

In [224]:
item_features = [ 
    'shop_item_share_of_shop_units_mean'
                ]

merge_col = ['item_id']
cols=item_features+merge_col

test = test.merge(training.drop_duplicates('item_id')[cols], on=merge_col, how='left')

In [106]:
shop_features = [
        'shop_me'
]

merge_col = ['shop_id']
cols=shop_features+merge_col


test = test.merge(training.drop_duplicates(merge_col)[cols], on=merge_col, how='left')

In [190]:
def add_rolls_test(df, cols, name, rolls = [3]):
    for roll in rolls:
        print(name, roll)
        roll_name = name+"_rolling_" + str(roll)
        roll_name_tmp = roll_name + "_tmp"
        
        try:
            df.drop(columns=[roll_name],inplace=True)
        except:
            pass       

    
        block_units_rolling_temp = training\
            .drop_duplicates(cols)\
            .sort_values(cols)\
            .set_index(cols)\
            .groupby(cols[0:len(cols)-1],as_index=False)\
            [name].rolling(roll,min_periods=2).mean().reset_index()\
            .rename(columns={name:roll_name})\
            [cols+[roll_name]]
        
        print([cols[0:len(cols)-1]+[roll_name]])
        thirty_three = block_units_rolling_temp[block_units_rolling_temp['date_block_num'] == 33].drop_duplicates(cols)\
                [cols[0:len(cols)-1]+[roll_name]]
        df = df.merge(thirty_three, on=cols[0:len(cols)-1], how='left')
    

        del block_units_rolling_temp
        gc.collect()
        

    
    return df
    

test = add_rolls_test(test, ['item_id','date_block_num'], 'item_block_mean')
test = add_rolls_test(test, ['shop_id','date_block_num'], 'shop_block_mean')
test = add_rolls_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_mean')


item_block_mean 3
[['item_id', 'item_block_mean_rolling_3']]
shop_block_mean 3
[['shop_id', 'shop_block_mean_rolling_3']]
shop_cat_block_mean 3
[['shop_id', 'item_category_id', 'shop_cat_block_mean_rolling_3']]


In [225]:
test = add_rolls_test(test, ['shop_id','item_id','date_block_num'], 'shop_item_block_mean')

shop_item_block_mean 3
[['shop_id', 'item_id', 'shop_item_block_mean_rolling_3']]


In [191]:
def add_lags_test(df, cols, name, lags = [1]):
    
    for lag in lags:
        print(name, lag)
        lag_name = name + "_lag_" + str(lag)
        
        try:
            df.drop(columns=[lag_name],inplace=True)
        except:
            pass       

        result = training\
            .drop_duplicates(cols)\
            .sort_values(cols)\
            .set_index(cols)\
            .groupby(cols[0:len(cols)-1],as_index=False)\
            [name].shift(lag)\
            .rename(columns={name:lag_name}).reset_index()
        
        thirty_three = result[result['date_block_num'] == 33].drop_duplicates(cols)\
                [cols[0:len(cols)-1] + [lag_name]]
        df = df.merge(thirty_three, on=cols[0:len(cols)-1], how='left')

        gc.collect()
    
    return df
                                         

                                        
test = add_lags_test(test, ['item_id','date_block_num'], 'item_block_mean')
test = add_lags_test(test, ['shop_id','date_block_num'], 'shop_block_mean')
test = add_lags_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_mean')


item_block_mean 1
shop_block_mean 1
shop_cat_block_mean 1


In [236]:
test = add_lags_test(test, ['shop_id','item_id','date_block_num'], 'shop_item_block_mean')

shop_item_block_mean 1


In [237]:
test.fillna(0, inplace=True)

In [239]:
test.sample(10)

,item_id,ID,shop_id,item_category_id,item_block_mean_rolling_3,shop_block_mean_rolling_3,shop_cat_block_mean_rolling_3,item_block_mean_lag_1,shop_block_mean_lag_1,shop_cat_block_mean_lag_1,item_minmax_q0.25,shop_item_share_of_shop_units_mean,shop_item_block_mean_rolling_3,shop_item_block_mean_lag_1
122898,13086,35513,10,62,0.000000,0.088107,0.031852,0.000000,0.098722,0.000000,8.75,0.016286,0.000000,0.0
40513,4141,129080,47,75,0.045719,0.334718,0.194790,0.023256,0.340216,0.083333,1.50,0.019063,0.000000,0.0
57333,5830,15353,3,65,0.659871,0.124087,0.158957,0.674419,0.132547,0.206897,45.75,0.018638,1.000000,3.0
203522,20832,167237,36,72,0.238431,0.000000,0.000000,0.139535,0.000000,0.000000,6.75,0.017852,0.000000,0.0
73628,7657,11602,6,55,0.639476,0.308454,0.298789,0.744186,0.315044,0.243789,31.00,0.019096,1.333333,2.0
134320,13667,22563,2,69,0.000000,0.158264,0.113065,0.000000,0.157915,0.056180,8.00,0.018474,0.000000,0.0
124498,13213,51744,26,47,0.580154,0.231153,0.155599,0.627907,0.238348,0.076190,14.50,0.019119,0.333333,1.0
136789,13880,189946,46,55,1.611774,0.260750,0.181349,1.325581,0.244248,0.167702,38.50,0.017504,0.666667,1.0
153187,15459,68096,24,63,0.085305,0.242064,0.178799,0.069767,0.277876,0.280000,9.25,0.016379,0.000000,0.0
74811,7792,46125,31,20,0.955770,1.005386,2.923999,1.209302,1.025959,3.205357,24.25,0.017852,1.000000,0.0


In [240]:
cb_preds = cb_model.predict(test[features])
cb_preds.clip(0,20,out=cb_preds)

array([0.07126889, 0.05115078, 0.17354015, ..., 0.14408137, 0.17389554,
       0.14687541])

In [241]:
np.max(cb_preds)

9.321474953496363

In [242]:
cb_preds[0:100]

array([0.07126889, 0.05115078, 0.17354015, 0.05168311, 0.04425439,
       0.05956398, 0.06654498, 0.03709658, 0.24925472, 0.56358786,
       0.07476183, 0.51388847, 0.04801934, 0.0781681 , 0.11834584,
       0.06260239, 0.0597869 , 0.04966577, 0.05263993, 0.0884839 ,
       0.29448249, 0.06037468, 0.04639789, 0.06735384, 0.06952595,
       0.06931253, 0.04090464, 0.29596081, 0.15624768, 0.04964671,
       0.03458684, 0.11744388, 0.23883286, 0.04365379, 0.08317746,
       0.04915596, 0.07702158, 0.08107208, 0.04621803, 0.06654815,
       0.04395338, 0.04704843, 0.10528114, 0.11637895, 0.52979822,
       0.0862661 , 0.12400927, 0.23035407, 0.17118923, 0.5335375 ,
       0.54881422, 0.90372061, 0.15318542, 0.85676016, 0.25403493,
       0.15774909, 0.36412968, 0.39089603, 0.08967485, 0.2850703 ,
       0.42456443, 0.22256631, 1.05920193, 0.21467825, 0.12146298,
       0.14104067, 0.12607403, 0.23389539, 0.36176052, 0.38091121,
       0.14108911, 0.29562199, 0.09664998, 0.12043529, 0.23625

In [243]:
submission = test.loc[:,['ID']]
submission['item_cnt_month'] = cb_preds

submission.to_csv('submission.csv', index=False)

In [205]:
training['shop_me_real']= training.groupby('shop_id')['shop_me'].transform(np.mean)
#training['item_me_real2']= training.groupby('item_id')['item_me'].transform('last')

In [217]:
training.groupby('item_id')['shop_item_share_of_shop_units'].transform(np.mean)



item_id
0        0.016472
1        0.015578
2        0.015766
3        0.015424
4        0.016472
5        0.015140
6        0.015126
7        0.015140
8        0.016090
9        0.015716
10       0.016289
11       0.015818
13       0.016472
14       0.015140
15       0.015818
16       0.016472
17       0.016472
18       0.015716
20       0.015716
21       0.016472
22       0.018262
23       0.016521
24       0.018262
25       0.015716
26       0.015788
27       0.014311
28       0.014002
29       0.014002
30       0.016379
31       0.016379
           ...   
22133    0.015356
22134    0.014002
22135    0.016223
22136    0.014254
22139    0.016379
22140    0.016133
22141    0.014444
22142    0.014335
22143    0.015302
22144    0.015736
22145    0.015516
22146    0.016146
22147    0.015063
22149    0.013630
22150    0.014587
22152    0.013630
22153    0.016116
22154    0.019119
22155    0.015554
22156    0.013243
22158    0.018474
22159    0.019049
22160    0.014968
22162    0.018025
22